In [1]:
!pip install transformers datasets torch scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!git clone https://github.com/jerryji1993/DNABERT.git

Cloning into 'DNABERT'...
remote: Enumerating objects: 778, done.
remote: Counting objects: 100% (778/778), done.
remote: Compressing objects: 100% (348/348), done.
remote: Total 778 (delta 420), reused 721 (delta 407), pack-reused 0 (from 0)
Receiving objects: 100% (778/778), 11.66 MiB | 18.99 MiB/s, done.
Resolving deltas: 100% (420/420), done.


In [3]:
%cd /content/DNABERT

/content/DNABERT


In [4]:
import pandas as pd
import os

# Directory containing the text files
directory = "/content/Data/"  # Change this to the correct path

# Function to parse FASTA format and assign labels
def parse_fasta(file_path, label):
    with open(file_path, "r") as file:
        data = file.read().split("\n")

    sequences = []
    current_seq = ""
    current_id = ""

    for line in data:
        if line.startswith(">"):  # Header line
            if current_seq:  # Save previous sequence
                sequences.append([current_id, current_seq, label])
            current_id = line.strip()  # Extract ID
            current_seq = ""  # Reset sequence
        else:
            current_seq += line.strip()  # Append sequence

    # Add last sequence
    if current_seq:
        sequences.append([current_id, current_seq, label])

    return sequences

# Process all .txt files in the directory
all_data = []
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        file_path = os.path.join(directory, filename)
        label = filename.replace(".txt", "").replace("_", " ").title()  # Extract disorder name
        all_data.extend(parse_fasta(file_path, label))

# Create DataFrame
df = pd.DataFrame(all_data, columns=["ID", "Sequence", "Disorder"])

# Save to CSV
output_path = os.path.join("/content", "genome_labeled_data.csv")
df.to_csv(output_path, index=False)
print(f"Saved labeled genome data to {output_path}")


Saved labeled genome data to /content/genome_labeled_data.csv


In [5]:
def kmer_tokenizer(sequence, k=6):
    """Splits DNA sequence into overlapping k-mers."""
    return [sequence[i:i+k] for i in range(len(sequence) - k + 1)]

# Apply k-mer tokenization
df["Tokens"] = df["Sequence"].apply(lambda x: " ".join(kmer_tokenizer(x, k=6)))

# Save tokenized data
df[["Tokens", "Disorder"]].to_csv("/content/tokenized_genome_data.csv", index=False)
print("Tokenized CSV file saved: tokenized_genome_data.csv")


Tokenized CSV file saved: tokenized_genome_data.csv


In [6]:
import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

# Load dataset
df = pd.read_csv("/content/tokenized_genome_data.csv")

# Encode labels
label_encoder = LabelEncoder()
df["Label"] = label_encoder.fit_transform(df["Disorder"])

# Split dataset
train_texts, val_texts, train_labels, val_labels = train_test_split(df["Tokens"], df["Label"], test_size=0.2, random_state=42)

# Load DNABERT tokenizer
tokenizer = BertTokenizer.from_pretrained("zhihan1996/DNA_bert_6")

# Tokenize sequences
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=512)

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_dict({
    "input_ids": train_encodings["input_ids"],
    "attention_mask": train_encodings["attention_mask"],
    "labels": list(train_labels),
})

val_dataset = Dataset.from_dict({
    "input_ids": val_encodings["input_ids"],
    "attention_mask": val_encodings["attention_mask"],
    "labels": list(val_labels),
})

# Load DNABERT model for classification
model = BertForSequenceClassification.from_pretrained("zhihan1996/DNA_bert_6", num_labels=len(df["Label"].unique()))




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/28.7k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/359M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at zhihan1996/DNA_bert_6 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

# Define training arguments
training_args = TrainingArguments(
    output_dir="/content/results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=25,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="/content/logs",
    logging_steps=10,
    save_total_limit=2,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train the model
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.081500,1.296654
2,1.124200,1.262555
3,1.043900,1.209477
4,1.025000,1.058926
5,0.689500,1.092807
6,0.491100,0.969817
7,0.450400,0.880412
8,0.283400,0.799719
9,0.250500,0.864033
10,0.112100,0.814702


In [8]:
# ✅ Save trained model
model.save_pretrained("/content/dnabert_finetuned")
tokenizer.save_pretrained("/content/dnabert_finetuned")

print("Model fine-tuned and saved successfully! 🎉")

Model fine-tuned and saved successfully! 🎉


In [9]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd

# Load saved model and tokenizer
model_path = "/content/dnabert_finetuned"
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)

# Set model to evaluation mode
model.eval()

# Load label encoder to decode predictions
df = pd.read_csv("/content/tokenized_genome_data.csv")
label_encoder = {i: label for i, label in enumerate(df["Disorder"].unique())}

def kmer_tokenizer(sequence, k=6):
    """Splits DNA sequence into overlapping k-mers."""
    return " ".join([sequence[i:i+k] for i in range(len(sequence) - k + 1)])

def predict_disorder(sequence):
    """Predicts the disorder based on the input DNA sequence."""
    # Tokenize the sequence into k-mers
    tokenized_seq = kmer_tokenizer(sequence, k=6)

    # Encode input sequence
    inputs = tokenizer(tokenized_seq, return_tensors="pt", truncation=True, padding=True, max_length=512)

    # Perform prediction
    with torch.no_grad():
        outputs = model(**inputs)

    # Get predicted label
    predicted_label = torch.argmax(outputs.logits, dim=1).item()

    return label_encoder.get(predicted_label, "Unknown Disorder")

# Example user input
user_sequence = """GGCTATGAGGTGGCTCCAGATGGGAGGACCTGTGTGGATATCAATGAATGTCTTCTAGAA
CCCAGAAAATGTGCACCAGGTACCTGTCAAAACTTGGATGGGTCCTACAGATGCATTTGC
CCACCTGGATACAGTCTTCAAAATGAGAAGTGTGAAGATATTGATGAGTGTGTCGAAGAG
CCAGAAATTTGTGCCCTGGGCACATGCAGTAACACTGAAGGCAGCTTCAAATGTCTGTGT
CCAGAAGGGTTTTCCTTGTCCTCCAGTGGAAGAAGGTGCCAAGATTTGCGAATGAGCTAC
TGTTATGCGAAGTTTGAAGGAGGAAAGTGTTCATCACCCAAATCCAGAAATCACTCCAAG
CAGGAATGCTGCTGTGCCTTGAAGGGAGAAGGCTGGGGAGACCCCTGCGAGCTCTGCCCC
ACGGAACCTGATGGTATGTCTGTCATCTGCATTTCTCTCTGGGCCATGCAGGGTGCAGAC
TGGCCATGATGTAAGTAAGACTGTAGGCCTTAAAGACAGTAACACAATTTAATTTAATTG
TAAGTCTAATTAGAATGTTCATTAAAAATTATTTTATCT"""
# input("Enter DNA sequence: ")
predicted_disorder = predict_disorder(user_sequence)

print(f"Predicted Disorder: {predicted_disorder}")


Predicted Disorder: Cystic Fibrosis


In [11]:
import torch
import torch.nn.functional as F
from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd

# Load saved model and tokenizer
model_path = "/content/dnabert_finetuned"
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)

# Set model to evaluation mode
model.eval()

# Load label encoder to decode predictions
df = pd.read_csv("/content/tokenized_genome_data.csv")
label_encoder = {i: label for i, label in enumerate(df["Disorder"].unique())}

def kmer_tokenizer(sequence, k=6):
    """Splits DNA sequence into overlapping k-mers."""
    return " ".join([sequence[i:i+k] for i in range(len(sequence) - k + 1)])

def predict_disorder(sequence):
    """Predicts the disorder based on the input DNA sequence and provides confidence score."""
    # Tokenize the sequence into k-mers
    tokenized_seq = kmer_tokenizer(sequence, k=6)

    # Encode input sequence
    inputs = tokenizer(tokenized_seq, return_tensors="pt", truncation=True, padding=True, max_length=512)

    # Perform prediction
    with torch.no_grad():
        outputs = model(**inputs)

    # Apply softmax to get probabilities
    probabilities = F.softmax(outputs.logits, dim=1).squeeze().tolist()

    # Get predicted class index and confidence score
    predicted_label_index = torch.argmax(outputs.logits, dim=1).item()
    confidence_score = probabilities[predicted_label_index] * 100  # Convert to percentage

    predicted_disorder = label_encoder.get(predicted_label_index, "Unknown Disorder")

    return predicted_disorder, confidence_score

# Example user input
# user_sequence = input("Enter DNA sequence: ")
user_sequence = """GGCTATGAGGTGGCTCCAGATGGGAGGACCTGTGTGGATATCAATGAATGTCTTCTAGAA
CCCAGAAAATGTGCACCAGGTACCTGTCAAAACTTGGATGGGTCCTACAGATGCATTTGC
CCACCTGGATACAGTCTTCAAAATGAGAAGTGTGAAGATATTGATGAGTGTGTCGAAGAG
CCAGAAATTTGTGCCCTGGGCACATGCAGTAACACTGAAGGCAGCTTCAAATGTCTGTGT
CCAGAAGGGTTTTCCTTGTCCTCCAGTGGAAGAAGGTGCCAAGATTTGCGAATGAGCTAC
TGTTATGCGAAGTTTGAAGGAGGAAAGTGTTCATCACCCAAATCCAGAAATCACTCCAAG
CAGGAATGCTGCTGTGCCTTGAAGGGAGAAGGCTGGGGAGACCCCTGCGAGCTCTGCCCC
ACGGAACCTGATGGTATGTCTGTCATCTGCATTTCTCTCTGGGCCATGCAGGGTGCAGAC
TGGCCATGATGTAAGTAAGACTGTAGGCCTTAAAGACAGTAACACAATTTAATTTAATTG
TAAGTCTAATTAGAATGTTCATTAAAAATTATTTTATCT"""
predicted_disorder, confidence = predict_disorder(user_sequence)

print(f"Predicted Disorder: {predicted_disorder} (Confidence: {confidence:.2f}%)")


Predicted Disorder: Cystic Fibrosis (Confidence: 32.60%)


In [20]:
!zip "/content/dnabert_finetuned.zip" -r "/content/dnabert_finetuned"

  adding: content/dnabert_finetuned/ (stored 0%)
  adding: content/dnabert_finetuned/config.json (deflated 62%)
  adding: content/dnabert_finetuned/vocab.txt (deflated 72%)
  adding: content/dnabert_finetuned/model.safetensors (deflated 7%)
  adding: content/dnabert_finetuned/tokenizer_config.json (deflated 75%)
  adding: content/dnabert_finetuned/special_tokens_map.json (deflated 42%)
